# Comment donner du sens à un fichier XML

## Présentation

Dans la mesure où le format XML est personnalisable, il devient essentiel de pouvoir contrôler la structure d’un fichier. Si certaines grammaires font consensus (HTML, TEI, TMX…), il est également possible de définir son propre schéma de validation.

L’une des manières de réaliser cette étape, la moins contraignante et la plus rapide à appréhender, est de passer par une DTD (*Document Type Definition*). Il s’agit d’un langage particulier, non-XML, qui offre des méthodes de contrôle des balises et des attributs en se passant de raffinements.

Au-delà d’être bien formé, tout fichier XML se doit également d’être valide.

### Déclarer une validation par DTD

Il existe deux façons de déclarer une DTD :

- soit en écrivant les règles de contrôle à l’intérieur même du fichier XML ;
- soit en faisant réféfence à un fichier externe (méthode privilégiée).

#### Déclaration interne

La liste des règles se présente dans une balise particulière `<!DOCTYPE>` qui prend pour nom l’étiquette de l’élément racine du document :

```xml
<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE maGrammaire [
    <!-- Liste des déclarations -->
]>
<maGrammaire/>
```

#### Déclaration externe

Pour faire référence à un fichier externe, il suffit d’indiquer le chemin vers la DTD dans une balise `<!DOCTYPE>` :

```xml
<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE maGrammaire SYSTEM "/chemin/vers/MaGrammaire.dtd">
```

**Remarque :** le nom de la grammaire porte toujours l’étiquette de l’élément racine.

## Déclarer des éléments

La syntaxe de déclaration d’un élément respecte le canon suivant :

```dtd
<!ELEMENT étiquette (modèle)>
```

Il existe plusieurs modèles de contenus :

- le **contenu d’élément** (seuls des éléments sont autorisés comme enfants de l’élément déclaré) ;
- le **contenu textuel** (seul du texte est attaché à l’élément) ;
- le **contenu mixte** (du texte et des éléments peuvent cohabiter) ;
- le **contenu vide** (l’élément est réputé autonome) ;
- le modèle de **contenu quelconque** (tout est autorisé).

### Le contenu d’élément

#### Principes généraux

L’élément déclaré ne peut contenir que d’autres éléments. À noter que l’ordre de déclaration des éléments enfants est signifiant :

```dtd
<!ELEMENT author (firstname, lastname)>
```

Cette déclaration signifie que les éléments `<firstname>` et `<lastname>`, sans présumer de leurs modèles propres, se succèdent dans cet ordre à l’intérieur de l’élément `<author>`, comme dans :

```xml
<author>
    <firstname/>
    <lastname/>
</author>
```

Des parenthèses peuvent servir à proposer des règles alternatives :

```dtd
<!ELEMENT author (
    (firstname, lastname) | name
)>
```

La déclaration précédente autoriserait en plus de la première la structure suivante :

```xml
<author>
    <name/>
</author>
```

#### Combinaisons

**Opérateurs logiques :** `,` `|`

**Quantificateurs :** `?` `*` `+`

```dtd
<!-- Sans suffixe : 'title' obligatoire une seule fois -->
<!ELEMENT head (title)>
<!-- Opérateur logique ET : 'title' et 'author' obligatoires -->
<!ELEMENT head (title, author)>
<!-- Opérateur logique OU : soit 'verse' soit 'line' -->
<!ELEMENT lyrics (verse | line)>
<!-- Quantificateur facultatif : 'duration' 0 ou 1 fois -->
<!ELEMENT head (title, author, duration?)>
<!-- Quantificateur facultatif multiple : 'verse' 0 ou n fois -->
<!ELEMENT lyrics (verse*)>
<!-- Quantificateur obligatoire : 'line' 1 fois ou plus -->
<!ELEMENT verse (line+)>
```

### Le contenu textuel

Ce modèle de contenu répond au cas de figure où seul du texte est attaché à un élément, comme dans :

```xml
<firstname>Pierre</firstname>
```

L’expression `#PCDATA` doit alors figurer comme seule possibilité :

```dtd
<!ELEMENT firstname (#PCDATA)>
```

### Le contenu mixte

La déclaration est plus permissive dans la mesure où, à l’intérieur d’un élément, peuvent cohabiter comme enfants directs du texte ou des éléments :

```xml
<author>
    <firstname>Pierre</firstname>
    Corneille
</author>
```

La règle pour valider la structure précédente doit mentionner l’expression `#PCDATA` en tête du modèle et moibliser l’opérateur logique **OU** entre chacun des éléments :

```dtd
<!ELEMENT author (#PCDATA | firstname)*>
```

**Remarque :** à éviter le plus possible pour des raisons de performance et d’optimisation.

### Le contenu vide

Dans le cas où l’élément est autonome, lorsqu’il est prévu pour ne comporter que des attributs par exemple, il convient de le déclarer avec la clause `EMPTY` :

```dtd
<!ELEMENT audio EMPTY>
```

### Le modèle de contenu quelconque

La clause `ANY` attachée à un élément permet de valider n’importe quelle structure. Il s’agit encore d’une structure à éviter autant que possible.

```dtd
<!ELEMENT audio ANY>
```

## Déclarer des attributs

Pour rappel, les attributs se placent dans la balise ouvrante d’un élément et se déclarent sous forme de paire `name="value"` :

```xml
<song author="Loreena McKennitt" title="The Highwayman"/>
```

La syntaxe de déclaration des attributs respecte le schéma suivant :

```dtd
<!ATTLIST étiquette
    attribut_1 TYPE clause
    attribut_2 TYPE clause
>
```

### Type textuel

Lorsque la valeur attribuée à un attribut ne revêt aucune importance particulière, il suffit de le déclarer comme texte simple :

```dtd
<!ELEMENT song EMPTY>
<!ATTLIST song
    author CDATA #REQUIRED
    title CDATA #REQUIRED
>
```

**Rappel !** Signes `<` `&` `>` `"` et `'` doivent être remplacés par une entité.

### Type identifiant

Assez souvent, le besoin apparaît d’identifier un élément de manière unique. Dans cette éventualité, on recourt à un attribut de type `ID` :

```dtd
<!ELEMENT verse (line+)>
<!ATTLIST verse
    num ID #REQUIRED
>
```

L’identifiant respecte les contraintes suivantes :
- il est unique dans tout le document ;
- aucun autre identifiant ne peut être attribué à un élément déjà identifié ;
- pour des raisons de clarté, sa valeur ne peut être partagée avec un autre élément ;
- il doit avoir un nom XML valide.

Les exemples ci-dessous sont par conséquent jugés illicites :

```xml
<!-- same value! -->
<verse num="n10">
    …
</verse>
<verse num="n10">
    …
</verse>
```

```xml
<!-- even in different elements, an ID can not appear twice -->
<verse vID="n10">
    …
    <line lID="n10">…</line>
</verse>
```

```xml
<!-- if author is an ID, validation will fail! -->
<verse num="v10" author="a1">
    …
</verse>
```



### Type référent d’identifiant

Afin de permettre de relier plusieurs éléments, il est possible de déclarer un référent d’identifiant :

```dtd
<!ELEMENT artist EMPTY>
<!ATTLIST artist
    tag ID #REQUIRED
>
<!ELEMENT song EMPTY>
<!ATTLIST song
    num ID #REQUIRED
    ref_artist IDREF #REQUIRED
>
```

Les déclarations précédentes permettent par exemple d’indiquer que l’élément identifié `loreena` est relié à l’élément identifié `c1` :

```xml
<artist tag="loreena"/>
<song num="c1" ref_artist="loreena"/>
```

### Type référent d’identifiants

Dans le cas où plusieurs éléments devraient être reliés à un autre, il existe le type `IDREFS` :

```dtd
<!ELEMENT artist EMPTY>
<!ATTLIST artist
    tag ID #REQUIRED
>
<!ELEMENT song EMPTY>
<!ATTLIST song
    num ID #REQUIRED
    artists IDREFS #REQUIRED
>
```

À présent, plusieurs artistes peuvent être reliés à une même chanson :

```xml
<artist tag="bowie"/>
<artist tag="jagger"/>
<song num="c2" artists="bowie jagger"/>
```

### Contrôler les valeurs permises

Afin d’obliger un attribut à prendre sa valeur parmi une liste contrôlée, il est possible de lister les possibilités en les combinants avec l’opérateur **OU** :

```dtd
<!ELEMENT paper EMPTY>
<!ATTLIST paper
    color (
        beige | yellow | dark
    ) "yellow"
>
```

Ainsi, les structures suivantes sont autorisées :

```xml
<paper color="beige"/>
<paper/>
```

**Remarque :** dans le deuxième cas, l’attribut `color` sera considéré comme présent et avec la valeur `yellow` affectée.

Sans surprise, l’apparition d’une valeur ne faisant pas partie de la liste contrôlée fera échouer la validation :

```xml
<paper color="red"/>
```

### Obliger ou non la présence d’un attribut

Les clauses `#REQUIRED` et `#IMPLIED` gèrent l’obligation de présence d’un attribut :

```dtd
<!ELEMENT paper EMPTY>
<!ATTLIST paper
    color (
        beige | yellow | black
    ) #REQUIRED
>
```

Dans ce cas de figure, l’attribut `color` devra être renseigné pour tout élément `paper` et devra en plus prendre sa valeur parmi la liste contrôlée.

Dans le cas ci-dessous, soit l’attribut `color` est présent et prend sa valeur parmi la liste des valeurs autorisées, soit il est permis qu’il soit absent :

```dtd
<!ELEMENT paper EMPTY>
<!ATTLIST paper
    color (
        beige | yellow | black
    ) #IMPLIED
>
```

### Imposer une valeur par défaut

La clause `#FIXED` permet d’imposer une valeur :

```dtd
<!ELEMENT paper EMPTY>
<!ATTLIST paper
    color CDATA #FIXED "beige"
>
```

Si l’attribut n’est pas présent, l’analyseur XML considérera qu’on lui a affecté la valeur `beige` et, s’il est au contraire renseigné, il devra bien comporter la valeur imposée au risque de faire échouer la validation.

## Les entités

Les entités permettent de remplacer une écriture formelle par du texte de substitution. Elles sont de quatre types :

- les entités prédéfinies ;
- les entités internes ;
- les entités externes ;
- les entités paramètres.

### Les entités prédéfinies

Le langage XML définit cinq entités :

|Caractère|Entité|Logogramme|Unicode|Unicode hexa
|:-|:-:|:-:|:-:|:-:|
|*Lower than*|`&lt;`|`<`|`&#60;`|`&#x3C;`|
|*Greater than*|`&gt;`|`>`|`&#62;`|`&#x3E;`|
|*Ampersand*|`&amp;`|`&`|`&#38;`|`&#x26;`|
|*Double quote*|`&quot;`|`"`|`&#34;`|`&#x22;`|
|*Apostroph*|`&apos;`|`'`|`&#39;`|`&#x27;`|

### Les entités internes

Aux entités prédéfinies, il est possible d’en déclarer de nouvelles :

```dtd
<!ENTITY bow "David Bowie">
```

L’entité `bow` peut ensuite être appelée directement dans le document XML :

```xml
<artist>&bow;</artist>
```

**Remarque :** une entité peut faire référence à une structure XML :

```dtd
<!ENTITY bow
    "<firstname>David</firstname>
    <lastname>Bowie</lastname>"
>
```

### Les entités externes

Sur le même principe, il peut parfois être pertinent de sauvegarder l’entité dans un fichier externe. Soit le fichier `bowie.xml` qui présente l’encodage suivant :

```xml
<?xml version="1.0" encoding="utf-8"?>
<artist>
    <firstname>David</firstname>
    <lastname>Bowie</lastname>
</artist>
```

Dans la DTD, on retrouve maintenant la déclaration de l’entité `bow` qui fait référence au fichier précédent :

```dtd
<!ENTITY bow SYSTEM "bowie.xml">
```

Dans le fichier validé par la DTD, il est désormais permis de simplement citer l’entité :

```xml
<album>
    &bow;
    <title>The Rise and Fall of Ziggy Stardust and the Spiders from Mars</title>
</album>
```

### Les entités paramètres

À la différence des entités classiques, les entités paramètres font référence à une déclaration DTD plutôt qu’à une structure XML.

Considérons l’entité paramètre suivante :

```dtd
<!ENTITY % identity
    "<!ELEMENT firstname (#PCDATA)>
    <!ELEMENT lastname (#PCDATA)>"
>
```

Plus loin dans la DTD, nous faisons référence à l’entité paramètre grâce au symbole `%` :

```dtd
<!ELEMENT artist (firstname?, lastname)>
    %identity;
```

## Valider un fichier XML

Un fichier XML est considéré valide s’il est à la fois **bien formé**, c’est-à-dire qu’il respecte les règles syntaxiques du format, et conforme à une grammaire de validation, qu’elle soit déclarée dans une DTD ou tout autre format.

### Vérifier la conformité syntaxique

La librairie Python `xml.etree.ElementTree` ne propose pas de méthode pour valider un document XML. Il faut soit utiliser une autre librairie, comme `lxml`, ou passer par un utilitaire en ligne de commande :

In [ ]:
! xmllint ./data/library.xml

L’option `--noout` permet d’éviter l’affichage du document XML :

In [ ]:
! xmllint --noout ./data/library.xml

Si un document XML n’est pas bien formé, la liste des erreurs ressort :

In [ ]:
! xmllint --noout ./data/faulty_catalog.xml

### Vérifier le respect des déclarations

Pour valider la grammaire d’un document XML, il faut rajouter l’option `--valid` dans la mesure où un fichier de validation a été attaché au document (par exemple par une déclaration `<!DOCTYPE>`) :

In [ ]:
! xmllint --noout --valid ./data/catalog.xml

**Remarque :** il convient de corriger les erreurs de syntaxe avant de procéder à la validation grammaticale.

Une fois la syntaxe corrigée, on peut également faire référence à une autre DTD que celle déclarée dans le document XML :

In [ ]:
! xmllint --noout --dtdvalid ./data/catalog.dtd ./data/catalog.xml